### 1 Vocabulary Creation

In [1]:
import pandas as pd

# Load the training data
columns = ['WordIndex', 'WordType', 'POSTag']
train_data = pd.read_csv('data/train', sep='\t', names=columns, quoting=3, skip_blank_lines=False)


In [2]:
word_counts = train_data['WordType'].value_counts()
print(word_counts)

WordType
,           46476
the         39533
.           37452
of          22104
to          21305
            ...  
Birthday        1
Happy           1
Bertie          1
crouched        1
Huricane        1
Name: count, Length: 43192, dtype: int64


In [3]:
rare_words = word_counts[word_counts < 3].index


In [4]:
train_data['WordType'] = train_data['WordType'].apply(lambda word: '<unk>' if word in rare_words else word)


In [5]:
updated_word_counts = train_data['WordType'].value_counts()
print(updated_word_counts)


WordType
,           46476
the         39533
.           37452
<unk>       32537
of          22104
            ...  
Portland        3
W               3
Ayer            3
stamp           3
Fiorini         3
Name: count, Length: 16919, dtype: int64


In [6]:
vocab = pd.DataFrame(updated_word_counts).reset_index()
vocab.columns = ['WordType', 'occurrences']

unk_row = vocab[vocab['WordType'] == '<unk>']
vocab = vocab[vocab['WordType'] != '<unk>']

vocab = vocab.sort_values(by='occurrences', ascending=False, kind='mergesort')

vocab = pd.concat([unk_row, vocab], ignore_index=True)
vocab['Index'] = range(len(vocab))
vocab[['WordType', 'Index', 'occurrences']].to_csv('vocab.txt', sep='\t', index=False, header=False)

total_occurrences_of_unk = vocab[vocab['WordType'] == '<unk>']['occurrences'].iloc[0]
print("Length of vocab:", len(vocab))
print("Total occurrences of <unk>:", total_occurrences_of_unk)

Length of vocab: 16919
Total occurrences of <unk>: 32537


### 2 Model learning

In [7]:
import pandas as pd
from collections import defaultdict
import json

In [8]:
#data from file is converted into a list of sentecnces where each sentence is represented as a list of (word, pos_tag) tuples

def load_data(filepath):
    sentences = []  # List to hold all sentences
    current_sentence = [] 
    with open(filepath, 'r') as file:
        for line in file:
            if line.strip():  # If not a blank line
                parts = line.split()
                index, word, pos_tag = parts[0], parts[1], parts[2]
                current_sentence.append((word, pos_tag))
            else:  # Sentence boundary
                if current_sentence:  # If current sentence is not empty
                    sentences.append(current_sentence)
                    current_sentence = []  # Reset for next sentence
    if current_sentence:  # Add last sentence if file does not end with a blank line
        sentences.append(current_sentence)
    return sentences

In [9]:
#initialized to hold integer counts

transition_counts = defaultdict(int)#count transitions between tags 
emission_counts = defaultdict(int)#count emissions of words from tags
tag_counts = defaultdict(int)#occurences of each tag

In [10]:
filepath = 'data/train' 
sentences = load_data(filepath)

In [11]:
for sentence in sentences:
    previous_tag = None
    for word, tag in sentence:
        #update emission count
        emission_counts[(tag, word)] += 1
        #update transition count
        if previous_tag is not None:
            transition_counts[(previous_tag, tag)] += 1
        #update tag count
        tag_counts[tag] += 1
        previous_tag = tag

In [12]:
#calculate transition and emission probabilities by dividing the counts by the total counts from each tag
#convert to strings for easy serialization to JSON, originally was a tuple.

transition_probs = {f'({k[0]},{k[1]})': v / tag_counts[k[0]] for k, v in transition_counts.items()}

emission_probs = {f'({k[0]},{k[1]})': v / tag_counts[k[0]] for k, v in emission_counts.items()}



In [13]:
# two keys
hmm_model = {
    "transition": transition_probs,
    "emission": emission_probs
}
with open('hmm.json', 'w') as f:
    json.dump(hmm_model, f, indent=4)#indentation of 4 spaces


In [14]:
print(f"Number of transition parameters: {len(transition_probs)}")
print(f"Number of emission parameters: {len(emission_probs)}")

Number of transition parameters: 1351
Number of emission parameters: 50286


In [15]:

print("Sample Transition Probabilities:")
for transition, prob in list(transition_probs.items())[:5]: 
    print(f"{transition}: {prob}")

print("\nSample Emission Probabilities:")
for emission, prob in list(emission_probs.items())[:5]: 
    print(f"{emission}: {prob}")

Sample Transition Probabilities:
(NNP,NNP): 0.3782645420509543
(NNP,,): 0.13846908958086018
(,,CD): 0.021234939759036144
(CD,NNS): 0.15775891730703062
(NNS,JJ): 0.017196978862406887

Sample Emission Probabilities:
(NNP,Pierre): 6.84868961738654e-05
(NNP,Vinken): 2.2828965391288468e-05
(,,,): 0.9999139414802065
(CD,61): 0.0007168253240050465
(NNS,years): 0.019530237301024905


In [16]:
# Assuming tag_counts is your dictionary containing tags as keys
tags_list = list(tag_counts.keys())  # Convert dictionary keys (tags) into a list

# Print the list of tags
print("List of Tags:", tags_list)


List of Tags: ['NNP', ',', 'CD', 'NNS', 'JJ', 'MD', 'VB', 'DT', 'NN', 'IN', '.', 'VBZ', 'VBG', 'CC', 'VBD', 'VBN', 'RB', 'TO', 'PRP', 'RBR', 'WDT', 'VBP', 'RP', 'PRP$', 'JJS', 'POS', '``', 'EX', "''", 'WP', ':', 'JJR', 'WRB', '$', 'NNPS', 'WP$', '-LRB-', '-RRB-', 'PDT', 'RBS', 'FW', 'UH', 'SYM', 'LS', '#']


In [17]:
total_words = sum(tag_counts.values())

# Print the total number of words
print(f"Total number of words: {total_words}")

Total number of words: 912095


### 3 Greedy decoding with HMM

In [18]:
import json

#load hmm model and extract the transition and emission probabilities.

with open('hmm.json', 'r') as f:
    hmm_model = json.load(f)

transition_probs = hmm_model['transition']
emission_probs = hmm_model['emission']


In [19]:
#assign each word, most probable tag using greedy approach.

#sentence -> list of words(the sentence) to tag.
#transition_probs -> dictionary of transition probabilities between tags
#emission_probs -> dictionary of emission probabilities of words given tags
#tag_counts -> occurances of each tag in the train data.

def greedy_decode(sentence, transition_probs, emission_probs, tag_counts):
    predicted_tags = []  
    previous_tag = None 
    
    for word in sentence: 
        #by default small probability is used to handle unseen word-tag pairs; same for tag-tag pairs.
        tag_prob = [(tag, emission_probs.get(f'({tag},{word})', 1e-6) * 
                   (transition_probs.get(f'({previous_tag},{tag})', 1e-6) if previous_tag else 1e-6))#setting transition probability as min probability for the first word; makes decision based on emission porbability alone. (uniform starting point; emission focus;) 
                    for tag in tag_counts]
        
        selected_tag = max(tag_prob, key=lambda x: x[1])[0]
        predicted_tags.append(selected_tag)
        previous_tag = selected_tag
    return predicted_tags

In [20]:
def load_sentences(filepath):
    sentences = []
    with open(filepath, 'r') as file:
        sentence = []  # Initialize an empty list for the current sentence
        for line in file:
            line = line.strip()
            if line:  # Non-empty line
                parts = line.split()
                word = parts[1]  #word is the second element
                sentence.append(word)
            else:  # Empty line indicates a sentence boundary
                if sentence:  # If current sentence is not empty
                    sentences.append(sentence)
                    sentence = []  # Reset for next sentence
        if sentence:  # Don't forget the last sentence
            sentences.append(sentence)
    return sentences

In [21]:
dev_filepath = 'data/dev'

#output_filepath: The path to the file where the predictions should be written.

def pred_tags_indices(sentences, transition_probs, emission_probs, tag_counts, output_filepath):
    with open(output_filepath, 'w') as out_file:
        for sentence in sentences:
            predicted_tags = greedy_decode(sentence, transition_probs, emission_probs, tag_counts)
            for index, (word, tag) in enumerate(zip(sentence, predicted_tags), start=1):
                out_file.write(f'{index} {word} {tag}\n')
            out_file.write('\n') 
            
            
dev_sentences = load_sentences(dev_filepath)
pred_tags_indices(dev_sentences, transition_probs, emission_probs, tag_counts, 'greedy_dev.out')



In [22]:
!python eval.py -p greedy_dev.out -g data/dev

total: 131768, correct: 123299, accuracy: 93.57%


In [23]:
test_filepath = 'data/test' 

test_sentences = load_sentences(test_filepath)

pred_tags_indices(test_sentences, transition_probs, emission_probs, tag_counts, 'greedy.out')


### 4 Viterbi decoding with HMM 

In [24]:
import json

#load hmm model and extract the transition and emission probabilities.

with open('hmm.json', 'r') as f:
    hmm_model = json.load(f)

transition_probs = hmm_model['transition']
emission_probs = hmm_model['emission']


In [25]:
def viterbi_decode(sentence, transition_probs, emission_probs, tag_counts):
    
    # dp[step][tag] = probability of the most probable path to each tag at that position and the tag from the previous position that leads to this probability
    dp = [{} for _ in sentence]
    
    #sets initial probabilities for each tag based on emission probabilities -> for first word in each sentence.
    for tag in tag_counts:
        dp[0][tag] = (emission_probs.get(f'({tag},{sentence[0]})', 1e-6), None)
    

    #forward pass
    #probability of the most probable path leading to this tag from any previous tag * transition probability from the previous tag to the current tag * the emission probability of the current word given the current tag 
    #max -> selects the prev tag that maximizes this probability ^
    for t in range(1, len(sentence)):
        for current_tag in tag_counts:
            max_prob, prev_tag = max(
                (dp[t-1][prev_tag][0] * transition_probs.get(f'({prev_tag},{current_tag})', 1e-6) * 
                 emission_probs.get(f'({current_tag},{sentence[t]})', 1e-6), prev_tag) 
                for prev_tag in tag_counts)
            dp[t][current_tag] = (max_prob, prev_tag)

            
    #Backtracking
    #selects the tag associated with the highest probability at the last step.
    #considered most probable final tag in sequence and then backtracks.
    last_word = len(sentence) - 1
    last_word_tag = max(dp[last_word], key=lambda tag: dp[last_word][tag][0])
    predicted_tags = [last_word_tag]

    
    #finding the most probable path
    for t in range(len(sentence) - 1, 0, -1):
        last_word_tag = dp[t][last_word_tag][1]
        predicted_tags.insert(0, last_word_tag)

    return predicted_tags 

In [26]:
def load_sentences(filepath):
    sentences = []
    with open(filepath, 'r') as file:
        sentence = []  # Initialize an empty list for the current sentence
        for line in file:
            line = line.strip()
            if line:  # Non-empty line
                parts = line.split()
                word = parts[1]  #the word is the second element
                sentence.append(word)
            else:  # Empty line indicates a sentence boundary
                if sentence:  # If current sentence is not empty
                    sentences.append(sentence)
                    sentence = []  # Reset for next sentence
        if sentence:  # Don't forget the last sentence
            sentences.append(sentence)
    return sentences

In [27]:
dev_filepath = 'data/dev'

def pred_tags_viterbi(sentences, transition_probs, emission_probs, tag_counts, output_filepath):
    with open(output_filepath, 'w') as out_file:
        for sentence in sentences:
            predicted_tags = viterbi_decode(sentence, transition_probs, emission_probs, tag_counts)
            for index, (word, tag) in enumerate(zip(sentence, predicted_tags), start=1):
                out_file.write(f'{index} {word} {tag}\n')
            out_file.write('\n')

            
dev_sentences = load_sentences(dev_filepath)
pred_tags_viterbi(dev_sentences, transition_probs, emission_probs, tag_counts, 'viterbi_dev.out')
   

In [28]:
!python eval.py -p viterbi_dev.out -g data/dev

total: 131768, correct: 124857, accuracy: 94.76%


In [29]:
test_filepath = 'data/test' 

test_sentences = load_sentences(test_filepath)

pred_tags_viterbi(test_sentences, transition_probs, emission_probs, tag_counts, 'viterbi.out')
